# Importações

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
import pandas as pd
from datetime import datetime

df_meteo = pd.read_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/open-meteo-51.49N0.16W23m.csv', sep=';')

df_incidents = pd.read_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/jan-dec-2022-gla-data-extract-attendant.csv')

In [85]:
df_meteo.head(100)

,time,temperature_2m (°C),relative_humidity_2m (%),precipitation (mm),rain (mm),snowfall (cm)
0,2022-01-01T00:00,12.7,93,0.0,0.0,0.0
1,2022-01-01T01:00,12.8,93,0.0,0.0,0.0
2,2022-01-01T02:00,12.5,93,0.0,0.0,0.0
3,2022-01-01T03:00,12.2,92,0.0,0.0,0.0
4,2022-01-01T04:00,11.9,93,0.0,0.0,0.0
...,...,...,...,...,...,...
95,2022-01-04T23:00,1.1,80,0.0,0.0,0.0
96,2022-01-05T00:00,1.5,78,0.0,0.0,0.0
97,2022-01-05T01:00,1.5,78,0.0,0.0,0.0
98,2022-01-05T02:00,1.5,76,0.0,0.0,0.0


In [86]:
df_incidents['Time'].head(100)

0     19:40
1     12:00
2     13:40
3     15:00
4     15:50
      ...  
95    20:40
96    03:45
97    08:59
98    02:12
99    17:30
Name: Time, Length: 100, dtype: object

# Date and time adjusts

## Weather dataset

In [87]:
df_meteo['date'] = pd.to_datetime(df_meteo['time']).dt.date
df_meteo['time'] = pd.to_datetime(df_meteo['time']).dt.time

In [88]:
df_meteo.head(100)

,time,temperature_2m (°C),relative_humidity_2m (%),precipitation (mm),rain (mm),snowfall (cm),date
0,00:00:00,12.7,93,0.0,0.0,0.0,2022-01-01
1,01:00:00,12.8,93,0.0,0.0,0.0,2022-01-01
2,02:00:00,12.5,93,0.0,0.0,0.0,2022-01-01
3,03:00:00,12.2,92,0.0,0.0,0.0,2022-01-01
4,04:00:00,11.9,93,0.0,0.0,0.0,2022-01-01
...,...,...,...,...,...,...,...
95,23:00:00,1.1,80,0.0,0.0,0.0,2022-01-04
96,00:00:00,1.5,78,0.0,0.0,0.0,2022-01-05
97,01:00:00,1.5,78,0.0,0.0,0.0,2022-01-05
98,02:00:00,1.5,76,0.0,0.0,0.0,2022-01-05


## Incident dataset

In [89]:
df_incidents['_Collision Date'] = pd.to_datetime(df_incidents['_Collision Date'])
df_incidents['Time'] = pd.to_datetime(df_incidents['Time']).dt.time

In [90]:
df_incidents['Time'].head(100)

0     19:40:00
1     12:00:00
2     13:40:00
3     15:00:00
4     15:50:00
        ...   
95    20:40:00
96    03:45:00
97    08:59:00
98    02:12:00
99    17:30:00
Name: Time, Length: 100, dtype: object

In [ ]:
df_incidents.head(100)

# Export datasets

In [92]:
df_incidents.to_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/incidents-data-2022.csv')
df_meteo.to_csv('/content/drive/MyDrive/Mestrado/WEB Data/Trabalho final/weather-data-2022.csv')